# dynamic 하게 example들을 선택하는 방법
정말 많은 예제들 중 어느 정도의 example들을 골라서 prompt에 허용할건지 정해야 한다.  
왜냐하면 많은 prompt는 더 많은 비용을 지불해야 하기 때문이다.  
비용이 있더라도 때로는 모델에 알맞는 양이 있기 때문. 




In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

먼저 다음 기능(LengthBasedExampleSelector) 을 import 해보자.


In [3]:
from langchain.prompts.example_selector import LengthBasedExampleSelector


LengthBasedExampleSelector 는 기본적으로 예제들을 형식화 할 수 있고,  
예제의 양이 얼마나 되는지 확인이 가능하다.  
그렇게 되면 우리가 설정해 놓은 세팅값에 따라 prompt에 알맞은 예제를 골라줄 것이다.  


In [4]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "country": "Greece",
        "answer": """
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "korea",
        "answer": """
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]

# 예제 대화 형식 지정
example_prompt = PromptTemplate.from_template("Human: {country}\nAI: {answer}")

이제 우리가 할 일은 FewShotPromptTemplate가 리스트에 있는 모든 예제들(examples)를 가지고 형식화 하기 전에 few_shot_chat_prompt 의 examples 를 지워준다.  
왜냐하면 지금 examples 는 모든 예제들이기 때문.  

example_selector 를 만들어서 사용해보자.

LengthBasedExampleSelector은 똑같이 examples를 필요로 한다.  
또, 형식화와 양이 얼마나 되는지 알아보기 위해서 example_prompt도 필요하다.  
예제의 양도 정해주자.  

In [7]:
# selector 만들기
example_selector = LengthBasedExampleSelector(
    # 사용할 예제들
    examples=examples,
    # 형식화와 양이 얼마나 되는지 알아보기 위한 example_prompt
    example_prompt=example_prompt,
    # 예제의 양을 얼마나 허용할지 정한다.(글자수?)
    max_length=100,
)

example_selector를 만들었으면 FewShotPromptTemplate 에 example_selector에 넣어주자.

In [8]:
few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples 대신 example_selector 사용
    example_selector=example_selector,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)

few_shot_prompt.format(country="austraila")


# chain = few_shot_prompt | chat

# chain.invoke({"country": "germany"})

'Human: Greece\nAI: \n        I know this: \n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about austraila'

위에서 볼 수 있듯 예제 수를 제한하는 것은 매우 유용하다.  
비용도 절약할 수 있다.  
또한 사용자의 질문이 항상 LLM으로 전달되어도 언제든지 예제들을 허용하는 양을 지키며 LLM에게 전달하게 될 것이다.  

---

전체 코드

In [10]:
# 전체 코드
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "country": "Greece",
        "answer": """
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "korea",
        "answer": """
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]

# 예제 대화 형식 지정
example_prompt = PromptTemplate.from_template("Human: {country}\nAI: {answer}")

# selector 만들기
example_selector = LengthBasedExampleSelector(
    # 사용할 예제들
    examples=examples,
    # 형식화와 양이 얼마나 되는지 알아보기 위한 example_prompt
    example_prompt=example_prompt,
    # 예제의 양을 얼마나 허용할지 정한다.(글자수?)
    max_length=180,
)

few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples 대신 example_selector 사용
    example_selector=example_selector,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)

few_shot_prompt.format(country="austraila")

'Human: Greece\nAI: \n        I know this: \n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: Italy\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about austraila'

# 자신만의 example selector 만드는 방법
random example selector을 구현해 보자. 
- 무작위로 선택하는 예제 선택기!




In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

# random example selector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "country": "Greece",
        "answer": """
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "korea",
        "answer": """
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        return self.examples.append(example)

    # 이 클래스는 select_example을 구현해 주어야 한다.
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


#  예제 대화 형식 지정
example_prompt = PromptTemplate.from_template("Human: {country}\nAI: {answer}")

# selector 만들기
example_selector = RandomExampleSelector(
    # 사용할 예제들
    examples=examples,
)

few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples 대신 example_selector 사용
    example_selector=example_selector,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)

few_shot_prompt.format(country="austraila")

'Human: Italy\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about austraila'

# Serialization and Composition
- Serialize(직렬화) 불러오기 저장 등을 의미한다.
- compose는 다양한 작은 prompt template을 가지고 이들을 모두 결합하는 것을 뜻한다.
---
이번에는 디스크에서 prompt templates을 가져오는 방법에 대해 배워보자.
먼저, 두 가지 타입의 prompt를 만들어야 한다.
하나는 Json 파일(prompt.json)이고, 다른 하나는 yaml(prompt.yaml) 파일이다.




prompt.json

``` json
{
    "_type": "prompt",
    "template": "what is the capital of {country}?",
    "input_variables": [
        "country"
    ]
}
```

json 파일 불러오기

In [30]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

prompt.format(country="Germany")

'what is the capital of Germany?'

다음은 yaml 파일 (prompt.yaml)

In [31]:
prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")

'what is the capital of Germany'

마지막으로 알아볼 것은 많은 prompt들의 memory등을 다 모으는 방법(다 합치는 방법)에 대해 알아보자. 

<b>PipelinePromptTemplate</b>

PipelinePromptTemplate은 많은 prompt들을 하나로 합칠 수 있도록 해준다.


In [32]:
from langchain.prompts.pipeline import PipelinePromptTemplate


4개의 prompt를 만들어 보자.  
3개는 작은 prompt들이고 마지막 1개는 모든 것을 포함하는 prompt가 될 것이다.  

첫 번째 prompt에 너는 role-playing 도우미이고 character을 흉내내는 AI라고 알려준다.

In [52]:
# 첫 번째 prompt
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character} say korean
"""
)

두 번째 prompt는 어떻게 캐릭터들이 말하는 지에 대한 예제들이 있다

어떻게 질문하는지 어떻게 답변하는지.

In [35]:
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

그리고 세 번째는 AI 도우미가 우리의 텍스트를 완성해주는 start 라는 예제


In [36]:
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You: 
"""
)

마지막으로, 이 모든 것들을 하나로 합친 prompt

In [53]:
final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

이제 모든 템플릿을 하나로 만들어 보자.

In [54]:
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

In [55]:
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

이렇게 작성하면 full_prompt 를 format 하면 유효성 검사 가능!



In [48]:
full_prompt.format(
    character="Pirate",
    example_question="what is your location?",
    example_answer="arrrrrr! that is secret!! arg arg!",
    question="what is your fav food?",
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n\n    \n    This is an example of how you talk:\n\n    Human: what is your location?\n    You: arrrrrr! that is secret!! arg arg!\n\n\n    \n    Start now!\n\n    Human: what is your fav food?\n    You: \n\n'

위 코드를 보면 롤 플레잉 도우미이고 해적을 흉내 내달라고 하는 소개용 prompt가 잘 들어간 걸 확인할 수 있다.  
어떻게 말하는 지에 대한 example 도,  

이것들이 바로 많은 prompt들을 하나의 prompt로 합치는 방법이다.

In [56]:
chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "what is your location?",
        "example_answer": "arrrrrr! that is secret!! arg arg!",
        "question": "what is your fav food?",
    }
)

Arrrrr! My favorite food be kimchi! Spicy and delicious, just like the treasures I seek on the high seas! Arrrrr!

AIMessageChunk(content='Arrrrr! My favorite food be kimchi! Spicy and delicious, just like the treasures I seek on the high seas! Arrrrr!')